In [3]:
!pip install pycaret==2.3.10 markupsafe==2.0.1 pyyaml==5.4.1 -qq
!pip uninstall scikit-learn -yx
!pip install scikit-learn==0.23.2


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -x
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score,roc_auc_score
from google.colab import drive
import warnings
drive.mount('/content/drive')

from pycaret.utils import enable_colab
import jinja2
enable_colab()
from pycaret.classification import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab mode enabled.


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# 새 섹션

In [32]:
#1. 내 드라이브에 SW_AI 폴더 생성 
#2. 공유드라이브의 Dataset폴더 바로가기를 자신의 드라이브 SW_AI폴더에 넣음
#3. 실행~_~
train  = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/train.csv')
test  = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/test.csv')

In [33]:
print(train.head())
print(train.shape)
print("Class counting")
print(train['nerdiness'].value_counts())
print(train.isnull().sum())

train_fill_nan = train.fillna(0)
print(train_fill_nan.shape)

   index   Q1   Q2   Q3   Q4   Q5   Q6   Q7   Q8   Q9  ...  engnat  age  hand  \
0      0  1.0  5.0  5.0  5.0  1.0  4.0  5.0  5.0  1.0  ...     1.0   20   2.0   
1      1  4.0  4.0  4.0  4.0  4.0  5.0  4.0  4.0  3.0  ...     1.0   49   1.0   
2      2  4.0  5.0  5.0  4.0  3.0  5.0  5.0  5.0  4.0  ...     2.0   43   1.0   
3      3  4.0  4.0  4.0  2.0  4.0  3.0  3.0  5.0  3.0  ...     1.0   17   2.0   
4      4  4.0  4.0  4.0  4.0  3.0  3.0  4.0  2.0  3.0  ...     2.0   18   2.0   

   religion  orientation  voted  married  familysize  ASD  nerdiness  
0      12.0          4.0    2.0      1.0         4.0  2.0          1  
1       2.0          1.0    1.0      2.0         4.0  2.0          1  
2       2.0          2.0    2.0      3.0         4.0  2.0          1  
3       1.0          1.0    2.0      1.0         2.0  2.0          1  
4      12.0          1.0    2.0      1.0         1.0  2.0          0  

[5 rows x 70 columns]
(15000, 70)
Class counting
1    8303
0    6697
Name: nerdiness, 

In [34]:
pd.set_option('display.max_rows', 100)
corr_df = train_fill_nan.corr(method='pearson')
print(corr_df['nerdiness'])

index           0.002742
Q1              0.280343
Q2              0.260365
Q3              0.201872
Q4              0.261849
Q5              0.232420
Q6              0.306619
Q7              0.197176
Q8              0.224878
Q9              0.218848
Q10             0.182959
Q11             0.216964
Q12             0.242044
Q13             0.215772
Q14             0.229456
Q15             0.250658
Q16             0.175394
Q17             0.203361
Q18             0.180605
Q19             0.158807
Q20             0.212169
Q21             0.161261
Q22             0.160440
Q23             0.200013
Q24             0.224117
Q25             0.177759
Q26             0.236975
introelapse     0.008319
testelapse      0.005550
surveyelapse    0.011953
TIPI1          -0.095200
TIPI2           0.032783
TIPI3           0.046240
TIPI4           0.054102
TIPI5           0.048562
TIPI6           0.100961
TIPI7           0.048928
TIPI8           0.044987
TIPI9          -0.006465
TIPI10         -0.086178


In [35]:
print(train_fill_nan)
print(train_fill_nan.columns)

       index   Q1   Q2   Q3   Q4   Q5   Q6   Q7   Q8   Q9  ...  engnat  age  \
0          0  1.0  5.0  5.0  5.0  1.0  4.0  5.0  5.0  1.0  ...     1.0   20   
1          1  4.0  4.0  4.0  4.0  4.0  5.0  4.0  4.0  3.0  ...     1.0   49   
2          2  4.0  5.0  5.0  4.0  3.0  5.0  5.0  5.0  4.0  ...     2.0   43   
3          3  4.0  4.0  4.0  2.0  4.0  3.0  3.0  5.0  3.0  ...     1.0   17   
4          4  4.0  4.0  4.0  4.0  3.0  3.0  4.0  2.0  3.0  ...     2.0   18   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...  ...   
14995  14995  2.0  5.0  4.0  3.0  3.0  4.0  4.0  4.0  3.0  ...     1.0   17   
14996  14996  5.0  4.0  5.0  4.0  4.0  5.0  5.0  4.0  4.0  ...     2.0   45   
14997  14997  4.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  4.0  ...     1.0   20   
14998  14998  5.0  5.0  4.0  5.0  5.0  5.0  5.0  1.0  5.0  ...     1.0   29   
14999  14999  5.0  4.0  2.0  5.0  2.0  2.0  4.0  2.0  4.0  ...     2.0   21   

       hand  religion  orientation  voted  married 

In [36]:
train_fill_nan

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,...,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [12]:
test

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,3.0,4.0,5.0,4.0,5.0,2.0,5.0,4.0,4.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0,CHL,9,114,243,2.0,4.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,1.0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,2.0,3,2.0,2.0,19,1.0,4.0,4.0,1.0,1.0,3.0,2.0
1,1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,5.0,4.0,5.0,USA,18,107,206,1.0,4.0,3.0,5.0,4.0,5.0,3.0,3.0,3.0,1.0,1,1,1,1,1,0,0,1,0,0,0,0,1,1,1,1,2.0,2,1.0,1.0,33,1.0,1.0,5.0,2.0,1.0,5.0,2.0
2,2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,1.0,5.0,5.0,4.0,4.0,3.0,4.0,5.0,2.0,3.0,5.0,4.0,5.0,5.0,1.0,5.0,IDN,4,87,191,3.0,3.0,4.0,3.0,4.0,4.0,5.0,3.0,2.0,1.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,1,1,1.0,3,2.0,2.0,13,1.0,4.0,5.0,2.0,1.0,3.0,2.0
3,3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,5.0,3.0,2.0,2.0,3.0,4.0,5.0,5.0,4.0,1.0,1.0,4.0,4.0,4.0,4.0,AUS,2409,155,258,1.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,1.0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,2.0,2,1.0,1.0,28,1.0,2.0,2.0,2.0,1.0,3.0,2.0
4,4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,5.0,5.0,4.0,1.0,5.0,2.0,3.0,5.0,5.0,3.0,3.0,PHL,24,163,289,3.0,3.0,3.0,5.0,5.0,4.0,3.0,4.0,3.0,4.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,2.0,3,2.0,2.0,15,1.0,4.0,5.0,2.0,1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,5.0,4.0,5.0,4.0,3.0,5.0,5.0,4.0,4.0,5.0,4.0,3.0,4.0,4.0,5.0,3.0,USA,10,200,254,3.0,5.0,5.0,5.0,5.0,3.0,5.0,3.0,3.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,2.0,3,2.0,2.0,16,3.0,10.0,4.0,2.0,1.0,3.0,2.0
35448,35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,4.0,4.0,3.0,5.0,GBR,504,136,155,3.0,3.0,5.0,5.0,3.0,5.0,5.0,3.0,1.0,2.0,1,0,0,1,1,0,0,1,0,1,0,0,1,1,1,1,2.0,2,3.0,1.0,16,3.0,2.0,5.0,2.0,1.0,2.0,2.0
35449,35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,1.0,5.0,5.0,4.0,5.0,USA,27,121,191,1.0,5.0,4.0,3.0,4.0,3.0,3.0,5.0,2.0,1.0,1,1,1,1,1,0,0,0,0,1,1,0,1,1,1,1,3.0,3,1.0,1.0,31,1.0,1.0,5.0,1.0,1.0,2.0,2.0
35450,35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,4.0,5.0,1.0,1.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,CAN,3,58,105,3.0,5.0,5.0,1.0,5.0,5.0,5.0,1.0,5.0,1.0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,2.0,3,1.0,1.0,19,1.0,12.0,1.0,2.0,1.0,NaN,2.0


In [37]:
print(test.shape)
print(test.isnull().sum())

(35452, 69)
index             0
Q1              119
Q2              147
Q3              130
Q4              125
Q5               90
Q6              132
Q7              142
Q8              108
Q9              104
Q10             220
Q11             119
Q12             149
Q13              96
Q14             102
Q15             107
Q16              85
Q17              79
Q18             147
Q19              97
Q20             119
Q21             123
Q22              82
Q23             114
Q24             125
Q25             108
Q26             108
country         450
introelapse       0
testelapse        0
surveyelapse      0
TIPI1           107
TIPI2           122
TIPI3           163
TIPI4           138
TIPI5           140
TIPI6           128
TIPI7           134
TIPI8           127
TIPI9           130
TIPI10          176
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9    

In [38]:
test = test.fillna(0)

In [39]:
train_fill_nan.drop(columns=['index','country','introelapse','testelapse','surveyelapse'],inplace=True)
test.drop(columns=['index','country','introelapse','testelapse','surveyelapse'], inplace=True)
print(test.shape)
print(train_fill_nan.shape)

(35452, 64)
(15000, 65)


In [40]:
exp_clf = setup(data = train_fill_nan, target = 'nerdiness', normalize = False,
                train_size=0.99, fold=5 ,silent=True, data_split_stratify=True, use_gpu=True,
                preprocess=True, session_id=123)
add_metric('auc_proba', 'AUC_PROBA', roc_auc_score, greater_is_better=True, target="pred_proba")

,Description,Value
0,session_id,123
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 65)"
5,Missing Values,False
6,Numeric Features,47
7,Categorical Features,17
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='nerdiness',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_s...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

Name                                                         AUC_PROBA
Display Name                                                 AUC_PROBA
Score Function              <function roc_auc_score at 0x7f06d04da5f0>
Scorer               make_scorer(roc_auc_score, needs_proba=True, e...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: auc_proba, dtype: object

In [50]:
best_5 = compare_models(sort='auc_proba', n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA,TT (Sec)
et,Extra Trees Classifier,0.7852,0.8756,0.8395,0.7867,0.8122,0.5618,0.5635,0.8756,2.934
rf,Random Forest Classifier,0.7887,0.8724,0.8444,0.7889,0.8157,0.5689,0.5708,0.8724,2.584
lightgbm,Light Gradient Boosting Machine,0.7508,0.8255,0.8101,0.7569,0.7825,0.4916,0.4932,0.8255,0.396
gbc,Gradient Boosting Classifier,0.7361,0.8067,0.7991,0.7434,0.7703,0.4613,0.4630,0.8067,3.296
lr,Logistic Regression,0.7263,0.7952,0.7922,0.7343,0.7621,0.4410,0.4428,0.7952,3.650
ada,Ada Boost Classifier,0.7271,0.7951,0.7840,0.7388,0.7608,0.4437,0.4448,0.7951,0.832
lda,Linear Discriminant Analysis,0.7261,0.7947,0.8025,0.7297,0.7644,0.4392,0.4420,0.7947,0.208
nb,Naive Bayes,0.6822,0.7393,0.7536,0.6970,0.7241,0.3507,0.3523,0.7393,0.036
knn,K Neighbors Classifier,0.6865,0.7386,0.7653,0.6977,0.7299,0.3583,0.3605,0.7386,5.226
dt,Decision Tree Classifier,0.7186,0.7149,0.7497,0.7441,0.7468,0.4302,0.4303,0.7149,0.186


INFO:logs:create_model_container: 30
INFO:logs:master_model_container: 30
INFO:logs:display_container: 5
INFO:logs:[ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
  

In [51]:
blended_l = stack_models(estimator_list = best_5[1:], meta_model= best_5[0],fold = 3, optimize = 'auc_proba')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA
Fold,,,,,,,,
0,0.7760,0.8626,0.8259,0.7817,0.8032,0.5436,0.5447,0.8626
1,0.7806,0.8668,0.8350,0.7830,0.8082,0.5526,0.5541,0.8668
2,0.7812,0.8658,0.8470,0.7775,0.8108,0.5525,0.5553,0.8658
Mean,0.7792,0.8651,0.8360,0.7807,0.8074,0.5496,0.5514,0.8651
Std,0.0023,0.0018,0.0086,0.0024,0.0031,0.0042,0.0047,0.0018


INFO:logs:create_model_container: 31
INFO:logs:master_model_container: 31
INFO:logs:display_container: 6
INFO:logs:StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
 

In [52]:
blended_tuned = tune_model(blended_l, optimize='auc_proba')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,AUC_PROBA
Fold,,,,,,,,
0,0.7832,0.8726,0.8060,0.8030,0.8045,0.5611,0.5611,0.8726
1,0.7896,0.8763,0.8084,0.8109,0.8096,0.5744,0.5744,0.8763
2,0.7946,0.8797,0.8090,0.8180,0.8135,0.5850,0.5851,0.8797
3,0.7842,0.8796,0.7914,0.8136,0.8023,0.5648,0.5650,0.8796
4,0.7878,0.8768,0.8089,0.8079,0.8084,0.5707,0.5707,0.8768
Mean,0.7879,0.8770,0.8047,0.8107,0.8077,0.5712,0.5713,0.8770
Std,0.0041,0.0026,0.0068,0.0051,0.0039,0.0083,0.0083,0.0026


INFO:logs:create_model_container: 32
INFO:logs:master_model_container: 32
INFO:logs:display_container: 7
INFO:logs:StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
 

In [53]:
model = finalize_model(blended_tuned)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                         

In [54]:
pred = predict_model(model, data=test, raw_score=True) 

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                           

In [56]:
pred['Score_1']

0        0.1110
1        0.8043
2        0.8826
3        0.5993
4        0.9314
          ...  
35447    0.9293
35448    0.6749
35449    0.9719
35450    0.0585
35451    0.6141
Name: Score_1, Length: 35452, dtype: float64

In [57]:
submission = pd.read_csv('./drive/MyDrive/SW_AI/Dataset/sample_submission.csv')
submission['nerdiness'] = pred['Score_1']
submission

,index,nerdiness
0,0,0.1110
1,1,0.8043
2,2,0.8826
3,3,0.5993
4,4,0.9314
...,...,...
35447,35447,0.9293
35448,35448,0.6749
35449,35449,0.9719
35450,35450,0.0585


In [58]:
submission.to_csv('./drive/MyDrive/SW_AI/Dataset/pycaret(testing_model5)_cain_submission.csv', index = False)